In [1]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import networkx as nx
from scipy.sparse.linalg import eigs, eigsh
from scipy.linalg import eig, eigh
from functools import reduce
import pickle
import qutip as qt
import jax
# from jax import config; config.update('jax_enable_x64', True) # config.update('jax_platform_name', 'cpu')
import jax.numpy as jnp

from openfermion.chem.molecular_data import spinorb_from_spatial
from openfermion.ops import InteractionOperator, FermionOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_sparse_operator, eigenspectrum

from pyscf import gto, dft, scf, cc, df, ao2mo, fci
# from pyscf import fci

import pennylane as qml
import pennylane.numpy as qmlnp
from pennylane import qchem

from ham import(
    pauli, get_qml_ham
)

from utils import(
    PauSumHam, taper, bit_count
)

from tapering import(
    get_ao_rep,
    get_generators
)

bohr = 0.529177249


## Structure

In [2]:
symbols = ["Be", "H", "H"]
d = 1.3
coordinates = qmlnp.array([[0., 0., 0.], [0., 0., -d], [0., 0., d]], requires_grad=False)
n_electrons = 6

## Hamiltonian

In [3]:
qham, n_qubits = qchem.molecular_hamiltonian(symbols, coordinates, basis='sto-3g', mapping='jordan_wigner')

### generators

In [4]:
generators = qml.symmetry_generators(qham)
paulixops = qml.paulix_ops(generators, n_qubits)
paulix_sector = qml.qchem.optimal_sector(qham, generators, n_electrons)

/home/physics/.local/lib/python3.8/site-packages/pennylane/operation.py:1830: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
/home/physics/.local/lib/python3.8/site-packages/pennylane/operation.py:1993: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(


### qham tapering

In [5]:
# qham_tapered = qml.taper(qham, generators, paulixops, paulix_sector)
# over 4 min, memory not enough

In [6]:
# E_qml_tapered = eigsh(qml.utils.sparse_hamiltonian(qham_tapered), k=1, which='SA')[0]

### psh ham tapering

In [7]:
# psham_tapered = taper(qham, generators, paulixops, paulix_sector)

In [8]:
# E_psh_tapered = eigsh(psham_tapered.to_sparse(), k=1, which='SA')[0][0] # take 11 min
E_psh_tapered = -14.82514221821951

In [9]:
E_psh_tapered

-14.82514221821951

In [10]:
# n_qubits, psham_tapered.n_qubits # (14, 10)

### point-group symmetry tapering (with psh)

In [11]:
# n_basis, params = qchem.mol_basis_data('sto-3g', symbols)
# alpha = qmlnp.array([params[i][1] for i in range(len(symbols))])
# alpha, n_basis, params

In [12]:
mol = qchem.Molecule(symbols, coordinates) # , alpha=alpha


In [13]:
qchem.scf(mol)() # alpha
mo_coeff = mol.mo_coefficients
n_ao = mo_coeff.shape[0]

In [14]:
# with open('BeH2_mo_coeff.pickle', 'rb') as f:
#     mo_coeff = pickle.load(f)

In [15]:
np.round(mo_coeff, 2)

tensor([[-0.98, -0.  ,  0.38,  0.  ,  0.  ,  0.2 , -0.  ],
        [ 0.02, -0.  , -0.43,  0.  ,  0.  ,  2.13, -0.  ],
        [-0.  ,  0.  ,  0.  ,  1.  ,  0.  , -0.  , -0.  ],
        [-0.  ,  0.  , -0.  ,  0.  , -1.  , -0.  , -0.  ],
        [-0.  , -0.45, -0.  , -0.  , -0.  ,  0.  ,  2.84],
        [-0.04,  0.48, -0.4 , -0.  , -0.  , -1.33,  2.4 ],
        [-0.04, -0.48, -0.4 , -0.  ,  0.  , -1.33, -2.4 ]], requires_grad=True)

In [16]:
with open('BeH2_ao_info.pickle', 'rb') as f:
    ao_info = pickle.load(f)

with open('BeH2_rotations.pickle', 'rb') as f:
    rotations = pickle.load(f)

# BeH2_rotations

In [17]:
ao_rep_list = get_ao_rep(n_ao, rotations, *ao_info, mol.coordinates)

In [18]:
sym_array = np.zeros(4, dtype=np.int32)
sym_array[2] == 1
sym_array

array([0, 0, 0, 0], dtype=int32)

In [27]:
def get_mo_sym(ao_rep_list, mo_coeff):
    N = len(ao_rep_list)
    M = mo_coeff.shape[1]

    sym_list = [] # np.empty((N, M), dtype=np.int32)
    for i in range(N):
        sym_array = np.empty(M, dtype=np.int32)
        sym_tag = True
        for j in range(M):
            org_vec = mo_coeff[:, j]
            rep_vec = ao_rep_list[i] @ org_vec
            # print(org_vec, rep_vec)
            # if   np.allclose(rep_vec,  org_vec, rtol=1e-3, atol=1e-6): sym_array[j] =  1
            # elif np.allclose(rep_vec, -org_vec, rtol=1e-3, atol=1e-6): sym_array[j] = -1
            if   (np.abs(rep_vec - org_vec) <= 1e-2).all(): sym_array[j] =  1
            elif (np.abs(rep_vec + org_vec) <= 1e-2).all(): sym_array[j] = -1
            else:
                # print(sym_array)
                sym_tag = False
                break
        if sym_tag:
            # print(sym_array)
            sym_list.append(sym_array)
    
    return sym_list

mo_sym_list = get_mo_sym(ao_rep_list, mo_coeff)
mo_sym_list

[array([ 1, -1,  1, -1, -1,  1, -1], dtype=int32),
 array([ 1,  1,  1, -1, -1,  1,  1], dtype=int32),
 array([ 1, -1,  1,  1,  1,  1, -1], dtype=int32),
 array([ 1, -1,  1,  1, -1,  1, -1], dtype=int32),
 array([ 1,  1,  1, -1,  1,  1,  1], dtype=int32),
 array([ 1, -1,  1, -1,  1,  1, -1], dtype=int32),
 array([ 1,  1,  1,  1, -1,  1,  1], dtype=int32)]

In [28]:
np.kron(mo_sym_list[0], np.array([1, 1]))

array([ 1,  1, -1, -1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1])

In [29]:
def get_kernel(ao_rep_list, mo_coeff):
    from pennylane.qchem.tapering import _reduced_row_echelon

    n_mo = mo_coeff.shape[1]
    mo_sym_list = get_mo_sym(ao_rep_list, mo_coeff)

    spin_arr = np.array([1, -1], dtype=np.int32)
    mo_arr = np.ones(n_mo, dtype=np.int32)
    spin_swap_list = [np.kron(mo_arr, spin_arr), np.kron(mo_arr, -spin_arr)]
    so_sym_list = spin_swap_list + [np.kron(mo_sym, np.ones(2, dtype=np.int32)) for mo_sym in mo_sym_list]

    so_sym_array = np.array(so_sym_list)
    kernel = np.where(so_sym_array == 1, 0, 1)
    kernel = _reduced_row_echelon(kernel)
    kernel = np.delete(kernel, np.where(np.sum(kernel, 1) == 0),axis=0)
    return kernel


In [30]:
kernel = get_kernel(ao_rep_list, mo_coeff)
kernel

array([[1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1],
       [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]])

In [31]:
generators = get_generators(kernel)
paulixops = qml.paulix_ops(generators, n_qubits)
paulix_sector = qml.qchem.optimal_sector(qham, generators, n_electrons)

In [32]:
psham_pgs_tapered = taper(qham, generators, paulixops, paulix_sector)

In [33]:
E_psh_pgs_tapered = eigsh(psham_pgs_tapered.to_sparse(), k=1, which='SA')[0][0] # take 11 min

In [34]:
E_psh_pgs_tapered, E_psh_tapered

(-14.825142218213395, -14.82514221821951)

In [36]:
n_qubits, psham_pgs_tapered.n_qubits

(14, 9)

In `get_mo_sym`, `tol` should be set to `1e-2` in order to taper 5 qubits; `tol = 1e-6` would only taper 4 qubits